In [3]:
import pandas as pd
import os

output_directory = r'C:\Users\abrau\uvic\seng474\project\CryptoAI\AndrewFiles'

# Load the processed CSV file into a DataFrame
processed_df = pd.read_csv(r'C:\Users\abrau\uvic\seng474\project\CryptoAI\AndrewFiles\XBTUSD_60.csv')

# Determine the split point (80% training, 20% test)
split_index = int(len(processed_df) * 0.8)

# Split the data
train_df = processed_df.iloc[:split_index]  # Top 80% for training
test_df = processed_df.iloc[split_index:]   # Bottom 20% for testing

# Save the training and test sets to separate CSV files
train_filepath = os.path.join(output_directory, "XBTUSD_60_train.csv")
test_filepath = os.path.join(output_directory, "XBTUSD_60_test.csv")

train_df.to_csv(train_filepath, index=False)
test_df.to_csv(test_filepath, index=False)

print(f"Training data saved to: {train_filepath}")
print(f"Test data saved to: {test_filepath}")

c:\Users\abrau\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Training data saved to: C:\Users\abrau\uvic\seng474\project\CryptoAI\AndrewFiles\XBTUSD_60_train.csv
Test data saved to: C:\Users\abrau\uvic\seng474\project\CryptoAI\AndrewFiles\XBTUSD_60_test.csv


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

# Load the training data
train_filepath = r'C:\Users\abrau\uvic\seng474\project\CryptoAI\AndrewFiles\XBTUSD_60_train.csv'
train_df = pd.read_csv(train_filepath)

# Extract relevant features (excluding Timestamp)
train_data = train_df.iloc[:, 1:].values  # OHLSVT columns

# Normalize data
scaler = MinMaxScaler()
train_data_scaled = scaler.fit_transform(train_data)

# Create sequences for LSTM
sequence_length = 50  # Use past 50 timesteps to predict next
X_train, y_train = [], []
for i in range(len(train_data_scaled) - sequence_length):
    X_train.append(train_data_scaled[i:i+sequence_length])
    y_train.append(train_data_scaled[i+sequence_length, 3])  # Predict closing price

X_train, y_train = np.array(X_train), np.array(y_train)

In [8]:
# Define LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(sequence_length, train_data.shape[1])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

# Print accuracy metrics
for key in history.history.keys():
    print(f"{key}: {history.history[key][-1]}")

Epoch 1/20


c:\Users\abrau\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1708/1708 ━━━━━━━━━━━━━━━━━━━━ 38s 20ms/step - loss: 9.3058e-04 - mae: 0.0126 - mse: 9.3058e-04 - val_loss: 0.0014 - val_mae: 0.0336 - val_mse: 0.0014
Epoch 2/20
1708/1708 ━━━━━━━━━━━━━━━━━━━━ 35s 20ms/step - loss: 1.5905e-04 - mae: 0.0067 - mse: 1.5905e-04 - val_loss: 2.0469e-04 - val_mae: 0.0093 - val_mse: 2.0469e-04
Epoch 3/20
1708/1708 ━━━━━━━━━━━━━━━━━━━━ 34s 20ms/step - loss: 1.3734e-04 - mae: 0.0063 - mse: 1.3734e-04 - val_loss: 0.0020 - val_mae: 0.0377 - val_mse: 0.0020
Epoch 4/20
1708/1708 ━━━━━━━━━━━━━━━━━━━━ 34s 20ms/step - loss: 1.1266e-04 - mae: 0.0058 - mse: 1.1266e-04 - val_loss: 0.0047 - val_mae: 0.0595 - val_mse: 0.0047
Epoch 5/20
1708/1708 ━━━━━━━━━━━━━━━━━━━━ 35s 20ms/step - loss: 1.1957e-04 - mae: 0.0057 - mse: 1.1957e-04 - val_loss: 0.0043 - val_mae: 0.0551 - val_mse: 0.0043
Epoch 6/20
1708/1708 ━━━━━━━━━━━━━━━━━━━━ 41s 24ms/step - loss: 8.6389e-05 - mae: 0.0050 - mse: 8.6389e-05 - val_loss: 0.0027 - val_mae: 0.0460 - val_mse: 0.0027
Epoch 7/20
1708/1708 ━━━━━━━━━━